In [10]:
# Libraries
import numpy as np   
import time
import scipy.sparse.linalg as spy
import warnings
warnings.filterwarnings('ignore')


def pos_def(n,x=5):
    A = np.random.rand(n,n)
    return (A+A.T) + x*np.eye(n)

n = 50
A = pos_def(n,x=6)
x_sol = np.floor(np.random.rand(n) * 100)
b = np.dot(A, x_sol)
print np.linalg.cond(A)
print (np.linalg.eig(A)[0] > 0).all()

102.323646705
True


In [15]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def prec_gmres(A_0,b_0,alpha,tol=1e-6,left = True):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = A + alpha*np.identity(b.shape[0])
    
    # Left preconditioner -> A and b changes
    if left:
        b = gmres(M,b,tol=tol)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        if left:
            w = gmres(M, np.dot(A,Q[:,i]),tol=tol)[0]
        else:
            w = np.dot(A, gmres(M, Q[:,i],tol=tol)[0])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        residual = np.linalg.norm(np.dot(H[:i+2,:i+1],y) - beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    if left:
        return x_tild, i+1
    else:
        return gmres(M, x_tild,tol=tol)[0],i+1

In [12]:
def trapezoid2(myfun, N, a, b):
    x = np.linspace(0, b, N/2) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0
    for i in xmiddle:
        int_val += 2*myfun(i).real
    int_val = 2*myfun(a).real + 2*int_val + 2*myfun(0)# + myfun(b)
    return 0.5*h*int_val

def z(t, c, r):
    return c + r*np.complex(np.cos(t), np.sin(t))

def dz(t, r):
    return r*np.complex(np.cos(t), np.sin(t))

def g(t,l,L,alpha,v,f, tol=1e-6):
    centro = (l + L)/2.
    radio = (L - l)/2.
    fz = f(z(t, centro, radio))
    dzz = dz(t, radio)
    p = fz*dzz
    # Separamos las matrices
    M = (centro + dzz - alpha)*np.identity(v.shape[0]) - A
    a = M.real
    b = M.imag
    al = v.real
    bet = v.imag
    
    # Construímos el nuevo sistema Hx = r
    H = np.zeros((a.shape[0]*2, a.shape[1]*2))
    H[:a.shape[0],:a.shape[1]] = a
    H[a.shape[0]:,a.shape[1]:] = a
    H[:a.shape[0],a.shape[1]:] = -b
    H[a.shape[0]:,:a.shape[1]] = b
    r = np.zeros((al.shape[0]*2))
    r[:al.shape[0]] = al
    r[al.shape[0]:] = bet
    
    sol = gmres(H,r,tol=tol)[0]
    gmr = sol[:(sol.shape[0]/2)] + 1j*sol[(sol.shape[0]/2):]
    #gmr = p*spy.gmres((centro + dzz - alpha)*np.identity(v.shape[0]) - A, v)[0]
    return p*gmr


def cauchy_integral(l, L, alpha, v, Nf = 50,N=128,tol=1e-6):
    #f = lambda x: (1. - (alpha/x)**(Nf+1))/(x - alpha)
    # Nueva función
    f = lambda x: (1. - (alpha/(2*x))**(Nf+1))/(x - alpha/2.)
    g1 = lambda t: g(t,l,L,alpha,v,f,tol=tol)
    val = trapezoid2(g1, N, -np.pi, np.pi) / (2.*np.pi)
    return val

In [13]:
# GMRes using contour integral to compute the preconditioner
def gmres(A_0, b_0, it=100, tol=1e-6, prec=False, left=True, alpha=0.0,aux_tol=1e-6):
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    A = np.copy(A_0)
    b = np.copy(b_0)
    
    if prec:
        l = alpha
        L = np.amax(np.sum(np.abs(A),axis=1))
        if left:
            b = cauchy_integral(l, L, alpha, b,tol=aux_tol)
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        if prec:
            if left:
                w = cauchy_integral(l,L,alpha,np.dot(A,Q[:,i]),tol=aux_tol)
            else:
                w = np.dot(A, cauchy_integral(l, L, alpha, Q[:,i],tol=aux_tol))
        else:
            w = np.dot(A,Q[:,i])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        residual = np.linalg.norm(np.dot(H[:i+2,:i+1],y) - beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    if left or not prec:
        return x_tild,i+1
    else:
        return cauchy_integral(l, L, alpha, x_tild,tol=aux_tol),i+1

In [14]:
alpha = 0.05
print "Exact solution:\n", x_sol
print "\nNp Solve:\n", spy.gmres(A,b)
print "\nNormal GMRes:\n", gmres(A,b)
print "\nLeft Prec NC:\n", prec_gmres(A,b,alpha)
#print "\nRight Prec NC:\n", prec_gmres(A,b,alpha,left=False)
print "\nCauchy Left Prec:\n", gmres(A, b,prec=True,alpha=alpha)
#print "\nCauchy Right Prec:\n", gmres(A,b,prec=True,left=False,alpha=alpha)

Exact solution:
[ 28.  38.  97.  83.  73.  47.  71.  88.   4.  58.  90.  23.  85.  57.  47.
  97.  85.  53.  24.   5.  29.  46.  92.  65.  50.  27.  58.  35.  25.   5.
  96.  40.  65.  26.  90.  88.  42.  30.  61.  43.  98.  42.  19.  46.   4.
  32.  58.  83.  35.   5.]

Np Solve:
(array([ 28.02587523,  38.03502917,  97.0259505 ,  82.98492964,
        73.01438163,  47.03534669,  70.95425885,  87.98850903,
         4.02360551,  57.97880391,  89.99522064,  23.04349656,
        85.09550261,  56.98148164,  47.00373208,  96.90725476,
        84.97146555,  53.0006912 ,  24.00599927,   4.98013972,
        28.98072908,  46.04002209,  91.99156967,  64.98509606,
        50.02649241,  27.014311  ,  57.99406483,  34.92923877,
        25.01617151,   4.97781142,  95.95076309,  40.0223138 ,
        65.02202836,  26.03261036,  89.95945592,  88.04362625,
        41.97415547,  30.06065037,  61.04307727,  42.94319832,
        98.02313656,  42.02427293,  19.01202772,  46.00129237,
         3.96649016,  31

In [18]:
all_h = []